In [ ]:
import spherpro.bro as spb
import spherpro.db as db
import re
import pathlib

# Aim

Remove the images that were manually determined bad from the valid images

In [ ]:
from src.config import Conf

In [ ]:
class V:
    COL_EXCLUDED_IMG = 'excluded_img_id'

class C(Conf):
    fol_bad = pathlib.Path('/mnt/bbvolume/server_homes/vitoz/mnt/tmp/scratch/vitoz/Data/Analysis/20200303_phys_p173-p176-physiology_v3/plots/qc_classification/bad')
    re_qc_fn = re.compile('qcplot_imgid_(?P<img_id>[0-9]+)')
    fn_datbad = fol_bad / 'dat_manualbad.csv'


In [ ]:
bro = spb.get_bro(C.fn_config)

In [ ]:
n_img = bro.session.query(db.valid_images.image_id).count()
n_obj = (bro.session.query(db.valid_objects.object_id)
            .join(db.objects)
            .filter(db.objects.object_type == 'cell').count())
print(n_img, n_obj)

In [ ]:
def get_id(fn, re):
    i = re.match(fn.stem).groups()[0]
    return int(i)

In [ ]:
bad_img_ids = [get_id(f, C.re_qc_fn) for f in C.fol_bad.glob('*.png')]

In [ ]:
len(bad_img_ids)

Get acsession, acid and cutid from 'bad' images

In [ ]:
q = (bro.session.query(db.images.acquisition_id,
                     db.images.image_id,
                     db.images.image_number,
                     db.images.crop_number,
                     db.acquisitions.acquisition_mcd_acid,
                     db.sites.site_mcd_panoramaid,
                     db.slideacs.slideac_name)
     .join(db.acquisitions, db.images.acquisition_id == db.acquisitions.acquisition_id)
     .join(db.sites)
     .join(db.slideacs)
     .filter(db.images.image_id.in_(bad_img_ids))
    )
dat_badimgs = bro.doquery(q)
dat_badimgs.head()

In [ ]:
dat_badimgs.to_csv(C.fn_datbad, index=False)